In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
googleAuth = GoogleAuth()
googleAuth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(googleAuth)

In [ ]:
file_path = 'https://drive.google.com/file/d/1pBVuyN2jSITNfJDvO9cgIxZIwLdkSTft/view?usp=sharing'
id = '1pBVuyN2jSITNfJDvO9cgIxZIwLdkSTft'
CSVfile = drive.CreateFile({'id':id}) 
CSVfile.GetContentFile('BDD-Citation.csv')

In [ ]:
import pandas as pd
import numpy as np
import re
import math
import string
import gensim
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
from pprint import pprint
from gensim import corpora, models
from gensim.utils import simple_preprocess, lemmatize
from gensim.parsing.preprocessing import STOPWORDS
from textblob import TextBlob, Word 

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def Process_Data(row_index, column_index):

  Context = str(reader.iloc[row_index][column_index])
  Context_without_ref = re.sub(r'{.+?}', ' ', Context)
  Context_without_number = re.sub(r'\d+', ' ', Context_without_ref.lower())
  Context_without_punct = Context_without_number.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
  Context_without_multiple_spaces = re.sub(' +', ' ', Context_without_punct)

  return Context_without_multiple_spaces


def TextBlob_Lemmatize(text):
    word = TextBlob(text)
    tag_dict = {"J": 'a', "N": 'n', 
                "V": 'v', "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in word.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return lemmatized_list[0]

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(stemmer.stem(TextBlob_Lemmatize(token)))
    return result

columns = ['ReferenceID', 'SourceID', 'ChapterNumber', 'ParagraphNumber',
           'SentenceNumber', 'Title', 'PublishedDate', 'Authors',
           'TextBeforeRefMention', 'TextWhereRefMention', 'TextAfterRefMention']

reader = pd.read_csv('BDD-Citation.csv', sep = ',', names = columns, encoding = 'ISO-8859-1', nrows = 9000)
Right_Contexts = pd.DataFrame(columns = ['Right_Contexts'])
Left_Contexts = pd.DataFrame(columns = ['Left_Contexts'])
Scientific_Paper = pd.DataFrame(columns = ['Scientific_Paper'])
stemmer = SnowballStemmer('english')


for i in range(reader.shape[0]):
  if str(reader.iloc[i][8]) != "":
    Left_Contexts.loc[reader.index[i]] = Process_Data(i, 8)
  if str(reader.iloc[i][10]) != "":
    Right_Contexts.loc[reader.index[i]] = Process_Data(i, 10)
  Scientific_Paper.loc[reader.index[i]] = Process_Data(i, 5) + " " + Process_Data(i, 8) + " " + Process_Data(i, 9) + " " + Process_Data(i, 10)

Left_Lemma = Left_Contexts['Left_Contexts'].map(preprocess)
Right_Lemma = Right_Contexts['Right_Contexts'].map(preprocess)
Paper_Lemma = Scientific_Paper['Scientific_Paper'].map(preprocess)


In [ ]:
!pip install pyLDAvis
from tqdm._tqdm_notebook import tnrange,tqdm
from gensim.models import CoherenceModel
from google.colab import files
import matplotlib.pyplot as plt
import pyLDAvis.gensim
import seaborn as sns
import time
import warnings


dictionary = gensim.corpora.Dictionary(Paper_Lemma)
bow_corpus = [dictionary.doc2bow(doc) for doc in Paper_Lemma]

Lda = models.LdaMulticore
coherenceList_umass = []
coherenceList_cv = []

num_topics_list = np.arange(30, 50, 2)
for num_topics in tqdm(num_topics_list):
    warnings.filterwarnings("ignore")
    lda = Lda(bow_corpus, num_topics = num_topics, id2word = dictionary, minimum_probability = 0, passes = 20)
    cm = CoherenceModel(model = lda, corpus = bow_corpus, dictionary = dictionary, coherence = 'u_mass')
    coherenceList_umass.append(cm.get_coherence())
    cm_cv = CoherenceModel(model = lda, corpus = bow_corpus, texts = Paper_Lemma, dictionary = dictionary, coherence = 'c_v')
    coherenceList_cv.append(cm_cv.get_coherence())
    vis = pyLDAvis.gensim.prepare(lda, bow_corpus, dictionary)

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics = num_topics_list[max_cv], alpha = 0.1, eta = 0.01, random_state = 123, id2word = dictionary, passes = 2, workers = 2)
for idx, topic in lda_model_tfidf.print_topics(-1):
  print('Topic: {} Word: {}'.format(idx, topic))

seuil, Candidate_Articles, First_Topic = 0, [], []
for i in range(0, reader.shape[0]):
  for index, score in sorted(lda_model_tfidf[bow_corpus[i]], key = lambda tup: -1*tup[1]):
    if seuil == 0: 
      Topic_Index = index
      First_Topic.append(index)
      break
    else:
      seuil += 1


test = input("Entrez un contexte : ")
print(First_Topic[int(test)])

for i in range(0, reader.shape[0]):
  Position = 0
  for index, score in sorted(lda_model_tfidf[bow_corpus[i]], key = lambda tup: -1*tup[1]):
    if Position == 3:
      break
    else:
      if index == First_Topic[int(test)]:
        Candidate_Articles.append(i)
      Position += 1


In [ ]:
from gensim.models import Word2Vec

def Word2Vec_Model(Lemma):
  model = Word2Vec(Lemma, size = 100, window = 5, sg = 0, min_count = 1)
  model.train(Lemma, total_examples = model.corpus_count, epochs = 100)
  sentence_list = []
  for sentence in Lemma:
    if not sentence: 
      sentence_list.append("null")
    else:
      word_list = []
      for word in sentence:
        word_list.append(model.wv[word])
      sentence_list.append(word_list)
  return sentence_list

Left_Sentences_List = Word2Vec_Model(Left_Lemma)
Right_Sentences_List = Word2Vec_Model(Right_Lemma)
Paper_Sentences_List = Word2Vec_Model(Paper_Lemma)

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


def Tensors_from_Word2Vec(Sentences_List):
  sentence_tensors_list = []
  for i in range(0, len(Sentences_List)):
    if Sentences_List[i] != 'null':
      sentence_tensors_list.append(torch.FloatTensor(list(map(list, zip(*Sentences_List[i])))))
    else:
      sentence_tensors_list.append("null")
  return sentence_tensors_list


Left_Tensors_List = Tensors_from_Word2Vec(Left_Sentences_List)
Right_Tensors_List = Tensors_from_Word2Vec(Right_Sentences_List)
Paper_Tensors_List = Tensors_from_Word2Vec(Paper_Sentences_List)

In [ ]:
FILTER_HEIGHT = 100

class CNN_Model(nn.Module):
  def __init__(self, FILTER_WIDTH):
    super(CNN_Model, self).__init__()
    self.conv = nn.Conv1d(1, 1, (FILTER_HEIGHT, FILTER_WIDTH), padding_mode = "zeros", padding = 20)

  def forward(self, x):
    out = self.conv(x)
    out = torch.tanh(out)
    return out


def CNN_Sentence_Representation(Tensors_List):
  sentences_representation_list = []
  for i in range(0, len(Paper_Tensors_List)):
    if Tensors_List[i] != 'null':
      sentence_features = torch.Tensor(0)
      for filter_width in range(2, 17):
        cnn = CNN_Model(filter_width)
        max_over_time_pool = torch.max(cnn(Tensors_List[i].unsqueeze(0).unsqueeze(0)))
        sentence_features = torch.cat((sentence_features, max_over_time_pool.unsqueeze(0)))
      sentences_representation_list.append(sentence_features.unsqueeze(0).unsqueeze(0))
    else:
      sentences_representation_list.append("null")
  return sentences_representation_list

def CNN_Sentence_Representation1(Tensors_List,test):
  sentences_representation_list = []
  if Tensors_List[test] != 'null':
    sentence_features = torch.Tensor(0)
    for filter_width in range(2, 17):
      cnn = CNN_Model(filter_width)
      max_over_time_pool = torch.max(cnn(Tensors_List[i].unsqueeze(0).unsqueeze(0)))
      sentence_features = torch.cat((sentence_features, max_over_time_pool.unsqueeze(0)))
    sentences_representation_list.append(sentence_features.unsqueeze(0).unsqueeze(0))
  else:
    sentences_representation_list.append("null")
  return sentences_representation_list

Left_CNN_Representation = CNN_Sentence_Representation1(Left_Tensors_List,int(test))
Right_CNN_Representation = CNN_Sentence_Representation1(Right_Tensors_List,int(test))
Paper_CNN_Representation = CNN_Sentence_Representation(Paper_Tensors_List)

In [ ]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, dropout):
      super(LSTM_Model, self).__init__()
      self.dropout = nn.Dropout(p = dropout)
      self.lstm = nn.LSTM(input_size, hidden_size, num_layers = 1, bidirectional = False)
      
    def forward(self, x):
      x = self.dropout(x)
      _outputs, (final_hidden, _final_cell) = self.lstm(x)
      return final_hidden.squeeze(0)

In [ ]:
import torch.optim as optim

def LSTM_Context_Representation(CNN_Representation, lstm, criterion):
  lstm_list = []
        
  for i in range(0, len(CNN_Representation)):
          
    if CNN_Representation[i] != 'null': 
      output = lstm(CNN_Representation[i])
      lstm_list.append(output)
    else:
      lstm_list.append("null")
  print('Finished Training') 
  return lstm_list


def LSTM_Paper_Representation(CNN_Representation, lstm):
  lstm_list = []
  
  for i in range(0, len(CNN_Representation)):
     if CNN_Representation[i] != 'null':
        output= lstm(CNN_Representation[i])
        lstm_list.append(output)

     else:
        lstm_list.append("null")
  return lstm_list


LSTM_Context = LSTM_Model(15, 15, 0.1)
Criterion_Context = nn.MSELoss()
LSTM_Paper = LSTM_Model(15, 30, 0.1)

Left_LSTM = LSTM_Context_Representation(Left_CNN_Representation, LSTM_Context, Criterion_Context)
Right_LSTM = LSTM_Context_Representation(Right_CNN_Representation, LSTM_Context, Criterion_Context)
Paper_LSTM = LSTM_Paper_Representation(Paper_CNN_Representation, LSTM_Paper)

Finished Training
Finished Training


In [ ]:
import torch.nn as nn

class MLP_Model(nn.Module):
  def __init__(self, size):
      super(MLP_Model, self).__init__()
      self.size = size
      self.layers = nn.Sequential(
          nn.Linear(size, size),
          nn.ReLU(),
          nn.Linear(size, size))
      
  def forward(self, x):
      x = self.layers(x)
      return x


In [ ]:
mlp = MLP_Model(30)
MLP_Contexts_List = []
for i in range(0, 1):
  print(i)
  Combined_Contexts = None
  if Left_LSTM[i] == 'null':
    Combined_Contexts = Right_LSTM[i]

  elif Right_LSTM[i] == 'null':
    Combined_Contexts = Left_LSTM[i]

  else:
    Combined_Contexts = torch.cat((Left_LSTM[i], Right_LSTM[i]), 1)

  MLP_Contexts_List.append(mlp(Combined_Contexts))

print(Candidate_Articles)

Relevance_Score = []

for i in range(0, reader.shape[0]):
  Relevance_Score.append(torch.matmul(Paper_LSTM[i], torch.transpose(MLP_Contexts_List[0], 0, 1)))

dictio = {}
for i in range(0, len(Candidate_Articles)):
  dictio[Candidate_Articles[i]] = Relevance_Score[Candidate_Articles[i]].item()
print({k: v for k, v in sorted(dictio.items(), key = lambda item: item[1], reverse = True)})

In [ ]:
print("le nombre d article correctement recommandées est: ", len(Candidate_Articles))

In [ ]:

MRR = 0

for i in range(1, len(Sorted_Dict) + 1):
  if reader.iloc[Sorted_Dict[i-1][0]][1] == reader.iloc[int(test)][1]:
    MRR = MRR + (1 / i)

MRR = MRR * (1 / len(Sorted_Dict))
print(MRR)

In [ ]:
dictio1 = {}
R=[10,20,30,50,80,100]
for i in range(0, len( Relevance_Score)):
  dictio1[i] = Relevance_Score[i].item()

list_key = []
Sorted_Dictio = {k: v for k, v in sorted(dictio1.items(), key = lambda item: item[1], reverse = True)}
for key in Sorted_Dictio.keys():
  list_key.append(key)

for element in R:
  p = 0
  for i in range(0, element):
    if list_key[i] in Candidate_Articles:
      p + = 1
  print("Avec un rappel =", element, ", le résultat est", (p / element))